In [1]:
# Load libraries
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set options for better display
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

In [2]:
# Read dataset using the DATA_PATH from .env
data_path = os.getenv("DATA_PATH")
df = pd.read_csv(data_path)

print(f"✅ Dataset loaded successfully: {data_path}")
print(f"Shape: {df.shape}")
df.head()

✅ Dataset loaded successfully: /Users/mickey/customer_purchase_behaviour/data/raw/dataset.csv
Shape: (3900, 18)


,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.10,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.10,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.10,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.50,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.70,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [3]:
# Overview of data
df.info()

# Quick stats for numeric columns
df.describe(include=[np.number])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3900 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,"3,900.00","3,900.00","3,900.00","3,900.00","3,900.00"
mean,"1,950.50",44.07,59.76,3.75,25.35
std,"1,125.98",15.21,23.69,0.72,14.45
min,1.00,18.00,20.00,2.50,1.00
25%,975.75,31.00,39.00,3.10,13.00
50%,"1,950.50",44.00,60.00,3.70,25.00
75%,"2,925.25",57.00,81.00,4.40,38.00
max,"3,900.00",70.00,100.00,5.00,50.00


In [4]:
# Count missing values
missing = df.isnull().sum()
print("Missing values per column:\n", missing[missing > 0])

# Check duplicates
duplicates = df.duplicated().sum()
print(f"\nTotal duplicate rows: {duplicates}")

Missing values per column:
 Series([], dtype: int64)

Total duplicate rows: 0


In [5]:
print("Column Names:\n", df.columns.tolist())

# Check a few unique values per column to understand data variety
for col in df.columns[:5]:
    print(f"\n{col} -> {df[col].unique()[:5]}")

Column Names:
 ['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category', 'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season', 'Review Rating', 'Subscription Status', 'Shipping Type', 'Discount Applied', 'Promo Code Used', 'Previous Purchases', 'Payment Method', 'Frequency of Purchases']

Customer ID -> [1 2 3 4 5]

Age -> [55 19 50 21 45]

Gender -> ['Male' 'Female']

Item Purchased -> ['Blouse' 'Sweater' 'Jeans' 'Sandals' 'Sneakers']

Category -> ['Clothing' 'Footwear' 'Outerwear' 'Accessories']


In [6]:
# Standardize column names to snake_case (lowercase with underscores)
df.columns = (
    df.columns
    .str.strip()              # remove extra spaces
    .str.lower()              # convert to lowercase
    .str.replace(' ', '_')    # replace spaces with underscores
    .str.replace(r'[^a-z0-9_]', '', regex=True)  # remove special chars
)

print("✅ Column names standardized:")
print(df.columns.tolist())

✅ Column names standardized:
['customer_id', 'age', 'gender', 'item_purchased', 'category', 'purchase_amount_usd', 'location', 'size', 'color', 'season', 'review_rating', 'subscription_status', 'shipping_type', 'discount_applied', 'promo_code_used', 'previous_purchases', 'payment_method', 'frequency_of_purchases']


In [7]:
# Strip whitespace from string columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype(str).str.strip()

print("✅ String columns trimmed for whitespace.")

✅ String columns trimmed for whitespace.


In [8]:
# Ensure purchase_amount_usd is numeric and non-negative
df['purchase_amount_usd'] = pd.to_numeric(df['purchase_amount_usd'], errors='coerce')
invalid_rows = df[df['purchase_amount_usd'] < 0].shape[0]
df = df[df['purchase_amount_usd'] >= 0]

print(f"✅ Removed {invalid_rows} invalid rows with negative purchase amounts.")


✅ Removed 0 invalid rows with negative purchase amounts.


In [9]:
# Ensure numeric types for proper analysis
df['age'] = df['age'].astype(int)
df['previous_purchases'] = df['previous_purchases'].astype(int)
df['review_rating'] = df['review_rating'].astype(float)

print("✅ Converted age, previous_purchases, and review_rating to numeric.")

✅ Converted age, previous_purchases, and review_rating to numeric.


In [11]:
# Save cleaned data for future use
cleaned_path = "/Users/mickey/customer_purchase_behaviour/data/processed/cleaned_customer_purchase.csv"
df.to_csv(cleaned_path, index=False)

print(f"✅ Cleaned dataset saved to: {cleaned_path}")
print("Final shape:", df.shape)

✅ Cleaned dataset saved to: /Users/mickey/customer_purchase_behaviour/data/processed/cleaned_customer_purchase.csv
Final shape: (3900, 18)


In [12]:
# Verify everything looks fine
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             3900 non-null   int64  
 1   age                     3900 non-null   int64  
 2   gender                  3900 non-null   object 
 3   item_purchased          3900 non-null   object 
 4   category                3900 non-null   object 
 5   purchase_amount_usd     3900 non-null   int64  
 6   location                3900 non-null   object 
 7   size                    3900 non-null   object 
 8   color                   3900 non-null   object 
 9   season                  3900 non-null   object 
 10  review_rating           3900 non-null   float64
 11  subscription_status     3900 non-null   object 
 12  shipping_type           3900 non-null   object 
 13  discount_applied        3900 non-null   object 
 14  promo_code_used         3900 non-null   